In [23]:
from src.embeddings import VietnameseEmbeddings
import glob
from langchain.storage import InMemoryByteStore
from langchain_community.vectorstores import DeepLake
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers import MultiVectorRetriever
from langchain_experimental.text_splitter import SemanticChunker

In [24]:
model_name="dangvantuan/vietnamese-embedding"
model_kwargs={'device': 'cpu'}

embedding = VietnameseEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

/home/hgiahuy/miniconda3/envs/rag-agent/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [25]:
data = glob.glob('data/**/*.txt', recursive=True)

In [26]:
data

['data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 9 Nhóm (lớp) môn học và điều kiện mở nhóm/Nhóm môn học.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 6 Kế hoạch giảng dạy và học tập/Kế hoạch giảng dạy và học tập.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 6 Kế hoạch giảng dạy và học tập/Kế hoạch học kỳ.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 6 Kế hoạch giảng dạy và học tập/Kế hoạch năm học.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 6 Kế hoạch giảng dạy và học tập/Thời khóa biểu.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 6 Kế hoạch giảng dạy và học tập/Thời gian học.txt',
 'data/ts2021-tro-ve-sau/Chương II. TỔ CHỨC ĐÀO TẠO/Điều 17 Cảnh báo kết quả học tập, buộc thôi học/Quy định về việc buộc thôi học.txt',
 'data/ts2021-tro-ve-sau/Chương I

In [27]:
loaders = [
    TextLoader(path) for path in data
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

In [28]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4000)
docs = text_splitter.split_documents(docs)

In [29]:


# The vectorstore to use to index the child chunks
vectorstore = DeepLake(embedding=embedding)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]

Deep Lake Dataset in ./deeplake/ already exists, loading from the storage


In [30]:
# The splitter to use to create smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

In [31]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)

In [32]:
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

Creating 6492 embeddings in 13 batches of size 500:: 100%|██████████| 13/13 [04:20<00:00, 20.03s/it]

Dataset(path='./deeplake/', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype       shape       dtype  compression
  -------    -------     -------     -------  ------- 
 embedding  embedding  (13099, 768)  float32   None   
    id        text      (13099, 1)     str     None   
 metadata     json      (13099, 1)     str     None   
   text       text      (13099, 1)     str     None   


In [21]:
retriever.vectorstore.similarity_search("Điểm đánh giá thành phần")[0]

Document(page_content='hình thức đánh giá thành phần và cách tính điểm tổng hợp: Việc lựa chọn các hình thức đánh giá thành phần và trọng số của các điểm đánh giá thành phần, cũng như cách tính điểm tổng hợp đánh giá môn', metadata={'source': 'data/ts2021-tro-ve-sau/Chương III. ĐÁNH GIÁ KẾT QUẢ HỌC TẬP/Điều 21 Đánh giá môn học/Quy định về hình thức đánh giá thành phần và cách tính điểm tổng hợp.txt', 'doc_id': '00150670-7035-4ff5-aa2b-34b163b2f6e7'})